In [68]:
import weaviate
import weaviate.classes as wvc
import os
import requests
import json
from unstructured.partition.pdf import partition_pdf
from pathlib import Path
import weaviate
from weaviate.embedded import EmbeddedOptions
import os
import weaviate.classes.config as wc
from weaviate.classes.init import AdditionalConfig, Timeout

In [69]:
#add streamlit frontend

In [70]:
client.close()

In [71]:
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

client = weaviate.connect_to_wcs(
    cluster_url=os.getenv("WCD_URL"),
    auth_credentials=weaviate.auth.AuthApiKey(os.getenv("WCD_API_KEY")),
    headers={
        "X-OpenAI-Api-Key": os.environ["OPENAI_APIKEY"]
    },
    additional_config=AdditionalConfig(timeout=Timeout(init=10, query=45, insert=120))
)


In [78]:
client.collections.delete_all()

collection_of_docs = client.collections.create(
    name="Document",
    properties=[
        wc.Property(name="content", data_type=wc.DataType.TEXT),
        wc.Property(name="source", data_type=wc.DataType.TEXT, skip_vectorization=True)
    ],
    vectorizer_config=wc.Configure.Vectorizer.text2vec_openai(),
    generative_config=wc.Configure.Generative.openai()
)

In [73]:
pdf_paths = ["Quickstart Competitive Analysis.pdf", "vector_research.pdf"]
#create a list of multiple pdfs
#create a new method to read multiple files using readfile then use that in insert_chunks_from_file

In [74]:
def get_chunks(s, maxlength):
    start = 0
    end = 0
    while start + maxlength  < len(s) and end != -1:
        end = s.rfind(" ", start, start + maxlength + 1)
        yield s[start:end]
        start = end +1
    yield s[start:]

In [75]:
def read_file(pdf_path):
    partitions = partition_pdf(pdf_path)
    pdf_text="\n".join([str(part) for part in partitions])
    return get_chunks(pdf_text, 500)



In [76]:
def insert_chunks_from_file(pdf_paths):
    
    for pdf_path in pdf_paths:
        chunks = read_file(pdf_path)
        chunks_list = []
        for chunk in chunks:
            chunks_list.append({
                "content": chunk,
                "source": "Quickstart Competitive Analysis.pdf"
            })

    

    docs = client.collections.get("Document")
    docs.data.insert_many(chunks_list)
    

In [77]:
insert_chunks_from_file(pdf_paths=pdf_paths)

/Users/armaan_thakker/Desktop/weaviate-test/myenv/lib/python3.8/site-packages/unstructured/partition/pdf.py:94: DeprecationWarning: The 'warn' method is deprecated, use 'warning' instead
  logger.warn(
detectron2 is not installed. Cannot use the hi_res partitioning strategy. Falling back to partitioning with the fast strategy.
detectron2 is not installed. Cannot use the hi_res partitioning strategy. Falling back to partitioning with the fast strategy.
